In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

# Unzip the dataset
!unzip /content/drive/MyDrive/BTP/dataset.zip -d /content

In [ ]:

# Create necessary directories
!mkdir -p /content/Dataset/labels
!mkdir -p /content/Dataset/images

In [ ]:
classes = ['head', 'helmet', 'person']

In [ ]:
import numpy

def convert_annotation(image_size, bbox):
    x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
    dw = np.float32(1. / int(image_size[0]))
    dh = np.float32(1. / int(image_size[1]))
    w, h = x2 - x1, y2 - y1
    x, y = x1 + (w / 2), y1 + (h / 2)  # Center
    x, y, w, h = x * dw, y * dh, w * dw, h * dh  # Normalization
    return [x, y, w, h]

In [ ]:
def save_annotation_txt(image_filename, image_size, bboxes):
    save_path = f'/content/Dataset/labels/{image_filename}.txt'
    with open(save_path, 'a+') as file:
        for bbox in bboxes:
            class_id = class_names.index(bbox[0])
            normalized_bbox = convert_annotation(image_size, bbox[1:])
            file.write(f"{class_id} {normalized_bbox[0]} {normalized_bbox[1]} {normalized_bbox[2]} {normalized_bbox[3]}\n")

In [ ]:
from xml.dom.minidom import parse

def process_xml_annotation(xml_folder_path, xml_filename):
    xml_path = f'{xml_folder_path}/{xml_filename}.xml'
    dom = parse(xml_path)
    root = dom.documentElement
    image_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    size_element = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_width = size_element.getElementsByTagName("width")[0].childNodes[0].data
    img_height = size_element.getElementsByTagName("height")[0].childNodes[0].data
    img_depth = size_element.getElementsByTagName("depth")[0].childNodes[0].data

    bboxes = []
    for obj in objects:
        class_name = obj.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(obj.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(obj.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(obj.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(obj.getElementsByTagName("ymax")[0].childNodes[0].data)
        bboxes.append([class_name, x1, y1, x2, y2])

    save_annotation_txt(xml_filename, [img_width, img_height], bboxes)

# List and process annotation files
annotation_files = os.listdir('/content/annotations')
for annotation_file in annotation_files:
    xml_filename = annotation_file.split(".")[0]
    process_xml_annotation('/content/annotations', xml_filename)

In [ ]:
from sklearn.model_selection import train_test_split
import os
import yaml

# List and split image files
image_files = os.listdir('/content/images')
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)

print('Total images:', len(image_files))
print('Train:', len(train_files))
print('Validation:', len(val_files))
print('Test:', len(test_files))


In [ ]:
from shutil import copyfile

def copy_files(file_list, labels_root, images_source, mode):
    image_dest_dir = Path(f'/content/Dataset/images/{mode}')
    if not image_dest_dir.exists():
        os.makedirs(image_dest_dir)

    label_dest_dir = Path(f'/content/Dataset/labels/{mode}')
    if not label_dest_dir.exists():
        os.makedirs(label_dest_dir)

    for file in file_list:
        image_name = file.replace('.png', '')
        image_source_path = f'{images_source}/{image_name}.png'
        label_source_path = f'{labels_root}/{image_name}.txt'

        # Copy image
        image_dest_path = f'/content/Dataset/images/{mode}/{image_name}.png'
        copyfile(image_source_path, image_dest_path)

        # Copy label
        label_dest_path = f'/content/Dataset/labels/{mode}/{image_name}.txt'
        copyfile(label_source_path, label_dest_path)

copy_files(train_files, '/content/Dataset/labels', '/content/images', "train")
copy_files(val_files, '/content/Dataset/labels', '/content/images', "val")
copy_files(test_files, '/content/Dataset/labels', '/content/images', "test")


In [ ]:
!ls /content/Dataset/images

In [ ]:
# Create configuration file
config = {
   "path": "/content/Dataset/images",
   "train": "train",
   "val": "val",
   "test": "test",
   "nc": 3,
   "names": class_names
}
with open("data.yaml", "w") as file:
   yaml.dump(config, file, default_flow_style=False)

In [ ]:
# Display the YAML file
!cat data.yaml

In [ ]:
pip install wandb

In [ ]:
!wandb disabled

In [ ]:
# Train YOLOv8 model
!yolo task=detect mode=train data=data.yaml model=yolov8s.pt epochs=5 lr0=0.001

In [ ]:
# Check the results
!ls /content/runs/detect/train

In [ ]:
# Validate the model
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=data.yaml

In [ ]:
# # Save the model to Google Drive
# model_path = "/content/runs/detect/train/weights/best.pt"
# drive_model_path = "/content/drive/MyDrive/best.pt"
# copyfile(model_path, drive_model_path)

# print(f"Model saved to {drive_model_path}")

In [ ]:
# from google.colab import files

# # Download the file
# files.download('/content/runs/detect/train/weights/best.pt')
